In [34]:
class Audio():
    def __init__(self):
        super(Audio, self).__init__()
        
        self.SHORT_NORMALIZE = (1.0/32768.0)
        self.RATE = 44100  
        self.INPUT_BLOCK_TIME = 0.05
        self.CHUNK = int(self.RATE*self.INPUT_BLOCK_TIME)
        self.CHANNEL = 2
        self.blocks = []
        
        self.microphone_name = "3- Andrea PureAudio"
        self.filename = ""

    def initiate_mic(self):
        self.pa = pyaudio.PyAudio()
        device_index = self.discover_mic()  
        self.mic = self.pa.open(format = pyaudio.paInt16,
                    channels = self.CHANNEL,
                    rate =  self.RATE ,
                    input = True,
                    input_device_index = device_index,
                    frames_per_buffer = self.CHUNK
                    )
        
    def discover_mic(self):

        for i in range(self.pa.get_device_count() ):     
            devinfo = self.pa.get_device_info_by_index(i)   

            if self.microphone_name in devinfo["name"]:
                    device_index = i
                    return device_index

    def record_audio(self, filename, runtime):

        self.blocks = []
        self.filename = filename

        print("Recording starts..")
        st = time.time()
        while True:
            block = self.mic.read(self.CHUNK)
            self.blocks.append(block)

            t = time.time() - st

            if t>runtime:               
                break
        print("Recording done!")
        
    def close_mic(self):
        self.mic.stop_stream()
        self.mic.close()
        self.pa.terminate()  

    def save_audio(self):
        wf = wave.open(self.filename, 'wb')
        wf.setnchannels(self.CHANNEL)
        wf.setsampwidth(self.pa.get_sample_size(pyaudio.paInt16))
        wf.setframerate(self.RATE)
        wf.writeframes(b''.join(self.blocks))
        wf.close()
        
        print("Saved as", self.filename)
        
    def load_audio(self):
        self.x, self.sr = librosa.load(self.filename)
    
    def tempogram(self):
        oenv = librosa.onset.onset_strength(y=self.x, sr=self.sr)
        tempogram = librosa.feature.tempogram(onset_envelope=oenv, sr=self.sr)       
        return tempogram

    def create_audio_features(self):
        feature = {}
        
        feature['spec_centroid'] = np.mean(librosa.feature.spectral_centroid(y=self.x, sr=self.sr))
        feature['spec_rolloff'] = np.mean(librosa.feature.spectral_rolloff(y=self.x, sr=self.sr))
        feature['spec_bandwidth'] = np.mean(librosa.feature.spectral_bandwidth(y=self.x, sr=self.sr))
        feature['rmse'] = np.mean(librosa.feature.rms(y=self.x))
        feature['chroma'] = np.mean(librosa.feature.chroma_stft(y=self.x, sr=self.sr))
        feature['tempogram'] = np.mean(self.tempogram())
        feature['max_amplitude'] = np.mean(np.max(self.x))
        feature['min_amplitude']  = np.mean(np.min(self.x))
        feature['mean_amplitude']  = np.mean(np.mean(self.x))
        feature['standard_deviation']  =  np.std(self.x)
        feature['variance']  = np.var(self.x)
        feature['skew']  = skew(self.x)
        feature['kurtosis']  = kurtosis(self.x)
        feature['zero_crossing_rate']  = sum(librosa.zero_crossings(self.x, pad=False))
        feature['quantiles_10']  = np.quantile(self.x, 0.10)
        feature['quantiles_25']  = np.quantile(self.x, 0.25)
        feature['quantiles_50']  = np.quantile(self.x, 0.50)
        feature['quantiles_75']  = np.quantile(self.x, 0.75)
        
        mfcc = librosa.feature.mfcc(y=self.x, sr=self.sr)
        for i in range(1, 20):
            feature[f'mfcc{i}'] = np.mean(mfcc[i])
        
        return feature